<a href="https://colab.research.google.com/github/Ritaban26/Content-Recommendation-System/blob/main/Content_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Definition & Objective**

##🎨Colour-Based Image Recommendation System.

##Problem Statement:-
In this rapidly expanding sea of digital design and visual art, professionals are confronted with an unprecedented volume of visual content across platforms such as Pinterest, Google Images, and Shutterstock. While these repositories offer vast resources, they create a significant challenge regarding information overload. Designers and artists frequently struggle to streamline their inspiration workflows, as locating specific imagery that aligns with a precise visual theme is time-consuming and inefficient.


##Real-world Relevance and Motivation:-
The motivation for this project stems from a practical workflow inefficiency encountered in my photography and web design. When I attempted to source images from an existing library to match a specific website theme (e.g., a green color palette), traditional search methods proved inadequate. This highlighted a distinct need for a system capable of filtering and recommending content based on specific color metrics.


## 1️⃣ Install Dependencies

In [1]:
%%capture
# Install all required packages
!pip install numpy>=1.24.0 \
             scikit-image>=0.21.0 \
             Pillow>=10.0.0 \
             faiss-cpu>=1.7.4 \
             scikit-learn>=1.3.0 \
             colorthief>=0.2.1 \
             ipywidgets>=8.0.0 \
             matplotlib>=3.7.0

print("✓ Dependencies installed!")

In [2]:
# Standard imports
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import sys
import os
import time
import logging

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)

# Configure logging
logging.basicConfig(level=logging.INFO)

print("✓ Standard imports complete!")
print(f"Python: {sys.version.split()[0]}")
print(f"NumPy: {np.__version__}")

✓ Standard imports complete!
Python: 3.12.12
NumPy: 2.0.2


## 2️⃣ Upload Your Images Folder

Upload a folder containing your images (supports: .jpg, .png, .bmp, .gif, .webp)

**Note:** In Colab, you can upload individual files but not folders directly.

### Option A: Upload ZIP of images (Recommended)
Create a ZIP of your images folder and upload it below.

### Option B: Mount Google Drive
If your images are in Google Drive, use the alternate cell.

In [3]:
# Upload ZIP of images
from google.colab import files
import zipfile

print("📤 Upload a ZIP file containing your images...")
uploaded = files.upload()

# Extract ZIP
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('images')
        print("✓ Extraction complete!")
        break

# Set image directory
IMAGE_DIR = 'images'

# List uploaded images
!ls -lh images/
print(f"\n✓ Images ready in: {IMAGE_DIR}")

📤 Upload a ZIP file containing your images...


Saving photos.zip to photos.zip

📦 Extracting photos.zip...
✓ Extraction complete!
total 4.0K
drwxr-xr-x 2 root root 4.0K Jan 13 19:49 photos

✓ Images ready in: images


In [ ]:
# ALTERNATIVE: Mount Google Drive (UNCOMMENT TO USE)
# from google.colab import drive
# drive.mount('/content/drive')
#
# # Set path to your images folder in Drive
# IMAGE_DIR = '/content/drive/MyDrive/your_images_folder'
# print(f"✓ Using images from: {IMAGE_DIR}")

## 3️⃣ Load Core Modules
### All Python code embedded below (no external files needed!)

In [4]:
# ============================================================================
# Feature Extraction Module
# ============================================================================

from colorthief import ColorThief
from skimage import color as skcolor
from sklearn.cluster import KMeans
from typing import List, Tuple, Dict, Optional
import io

class ColorFeatureExtractor:
    """Extracts color features from images for similarity search."""

    def __init__(self, n_colors: int = 5, quality: int = 10):
        self.n_colors = n_colors
        self.quality = quality

    def extract_dominant_colors(self, image: Image.Image) -> List[Tuple[int, int, int]]:
        """Extract dominant colors using k-means clustering."""
        try:
            img_array = np.array(image)
            pixels = img_array.reshape(-1, 3)

            if len(pixels) > 10000:
                indices = np.random.choice(len(pixels), 10000, replace=False)
                pixels = pixels[indices]

            kmeans = KMeans(
                n_clusters=self.n_colors,
                random_state=42,
                n_init=10,
                max_iter=500,
                tol=1e-6
            )
            kmeans.fit(pixels)

            colors = kmeans.cluster_centers_.astype(int)
            labels = kmeans.labels_
            unique, counts = np.unique(labels, return_counts=True)
            sorted_indices = np.argsort(-counts)
            colors = colors[sorted_indices]

            return [tuple(color) for color in colors]
        except Exception as e:
            return [(128, 128, 128)] * self.n_colors

    def rgb_to_lab(self, rgb: Tuple[int, int, int]) -> Tuple[float, float, float]:
        """Convert RGB to LAB color space."""
        rgb_normalized = np.array([[rgb]]) / 255.0
        lab = skcolor.rgb2lab(rgb_normalized)
        return tuple(lab[0, 0])

    def create_feature_vector(self, image: Image.Image) -> np.ndarray:
        """Create feature vector from image."""
        rgb_colors = self.extract_dominant_colors(image)
        lab_colors = [self.rgb_to_lab(rgb) for rgb in rgb_colors]
        feature_vector = np.array(lab_colors).flatten()
        return feature_vector.astype(np.float32)

    def extract_color_palette(self, image: Image.Image) -> Dict:
        """Extract color palette with RGB and LAB representations."""
        rgb_colors = self.extract_dominant_colors(image)
        lab_colors = [self.rgb_to_lab(rgb) for rgb in rgb_colors]
        hex_colors = [rgb_to_hex(rgb) for rgb in rgb_colors]

        return {
            'rgb_colors': rgb_colors,
            'lab_colors': lab_colors,
            'hex_colors': hex_colors,
            'n_colors': len(rgb_colors)
        }

def hex_to_rgb(hex_color: str) -> Tuple[int, int, int]:
    """Convert HEX to RGB."""
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb: Tuple[int, int, int]) -> str:
    """Convert RGB to HEX."""
    return '#{:02x}{:02x}{:02x}'.format(*rgb).upper()

print("✓ ColorFeatureExtractor loaded!")

✓ ColorFeatureExtractor loaded!


In [5]:
# ============================================================================
# Data Ingestion Module
# ============================================================================

logger = logging.getLogger(__name__)

class ImageDataset:
    """Manages image collection from directory."""

    def __init__(self, image_dir: str):
        self.image_dir = Path(image_dir)
        self.images: List[Dict] = []
        self.supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp'}

        if not self.image_dir.exists():
            raise ValueError(f"Directory does not exist: {image_dir}")

    def load_images(self, recursive: bool = True, max_size: Optional[tuple] = None) -> List[Dict]:
        """Load all images from directory."""
        logger.info(f"Loading images from {self.image_dir}")

        if recursive:
            image_paths = []
            for ext in self.supported_formats:
                image_paths.extend(self.image_dir.rglob(f"*{ext}"))
                image_paths.extend(self.image_dir.rglob(f"*{ext.upper()}"))
        else:
            image_paths = []
            for ext in self.supported_formats:
                image_paths.extend(self.image_dir.glob(f"*{ext}"))
                image_paths.extend(self.image_dir.glob(f"*{ext.upper()}"))

        image_paths = sorted(set(image_paths))
        logger.info(f"Found {len(image_paths)} image files")

        for img_path in image_paths:
            try:
                img = Image.open(img_path)

                if img.mode != 'RGB':
                    img = img.convert('RGB')

                if max_size:
                    img.thumbnail(max_size, Image.Resampling.LANCZOS)

                self.images.append({
                    'path': str(img_path),
                    'filename': img_path.name,
                    'size': img.size,
                    'format': img.format,
                    'image': img
                })
            except Exception as e:
                logger.warning(f"Failed to load {img_path}: {e}")
                continue

        logger.info(f"Successfully loaded {len(self.images)} images")
        return self.images

    def get_statistics(self) -> Dict:
        """Get dataset statistics."""
        if not self.images:
            return {}

        return {
            'total_images': len(self.images),
            'directory': str(self.image_dir)
        }

def load_images_from_directory(directory: str, recursive: bool = True,
                               max_size: Optional[tuple] = (800, 800)) -> ImageDataset:
    """Load images from directory."""
    dataset = ImageDataset(directory)
    dataset.load_images(recursive=recursive, max_size=max_size)
    return dataset

print("✓ ImageDataset loaded!")

✓ ImageDataset loaded!


In [6]:
# ============================================================================
# Indexing Module (FAISS)
# ============================================================================

import faiss
import pickle
import json

logger = logging.getLogger(__name__)

class ImageIndex:
    """Manages FAISS index for image similarity search."""

    def __init__(self, dimension: int = 15):
        self.dimension = dimension
        self.index = None
        self.metadata = []
        self.is_trained = False

    def build_index(self, feature_vectors: np.ndarray, metadata: List[Dict],
                   index_type: str = 'flat') -> None:
        """Build FAISS index from feature vectors."""
        if len(feature_vectors) != len(metadata):
            raise ValueError("Number of feature vectors must match metadata length")

        logger.info(f"Building FAISS index with {len(feature_vectors)} vectors")

        feature_vectors = feature_vectors.astype(np.float32)

        if index_type == 'flat':
            self.index = faiss.IndexFlatL2(self.dimension)
        else:
            raise ValueError(f"Unsupported index type: {index_type}")

        self.index.add(feature_vectors)
        self.metadata = metadata
        self.is_trained = True

        logger.info(f"Index built successfully with {self.index.ntotal} vectors")

    def search(self, query_vector: np.ndarray, k: int = 10) -> Tuple[np.ndarray, np.ndarray]:
        """Search for k nearest neighbors."""
        if not self.is_trained:
            raise ValueError("Index must be built before searching")

        if query_vector.ndim == 1:
            query_vector = query_vector.reshape(1, -1)
        query_vector = query_vector.astype(np.float32)

        k = min(k, self.index.ntotal)
        distances, indices = self.index.search(query_vector, k)

        return distances[0], indices[0]

    def get_statistics(self) -> Dict:
        """Get index statistics."""
        return {
            'dimension': self.dimension,
            'total_vectors': self.index.ntotal if self.index else 0,
            'is_trained': self.is_trained,
            'total_metadata': len(self.metadata)
        }

print("✓ ImageIndex loaded!")

✓ ImageIndex loaded!


In [7]:
# ============================================================================
# Similarity Search Module
# ============================================================================

from typing import Union

logger = logging.getLogger(__name__)

class ColorQueryEngine:
    """Query engine for color-based image similarity search."""

    def __init__(self, index: ImageIndex, extractor: ColorFeatureExtractor):
        self.index = index
        self.extractor = extractor

    def parse_color_input(self, color_input: Union[str, Tuple, List]) -> Tuple[int, int, int]:
        """Parse color input to RGB."""
        if isinstance(color_input, str):
            return hex_to_rgb(color_input)
        elif isinstance(color_input, (tuple, list)):
            if len(color_input) == 3:
                return tuple(int(c) for c in color_input)
            else:
                raise ValueError("RGB input must have 3 values")
        else:
            raise ValueError(f"Unsupported color input type: {type(color_input)}")

    def create_query_vector_from_color(self, color_input: Union[str, Tuple, List]) -> np.ndarray:
        """Create query vector from a single color."""
        rgb = self.parse_color_input(color_input)
        lab = self.extractor.rgb_to_lab(rgb)
        feature_vector = np.array([lab] * self.extractor.n_colors).flatten()
        return feature_vector.astype(np.float32)

    def query_by_color(self, color_input: Union[str, Tuple, List], k: int = 10) -> List[Dict]:
        """Find images similar to a given color."""
        logger.info(f"Querying by color: {color_input}")

        query_vector = self.create_query_vector_from_color(color_input)
        search_k = min(k * 5, self.index.index.ntotal)
        distances, indices = self.index.search(query_vector, search_k)

        results = []
        for dist, idx in zip(distances, indices):
            metadata = self.index.metadata[idx].copy()
            similarity = self.calculate_similarity_score(dist)

            if similarity >= 0.70:
                metadata['similarity_score'] = similarity
                metadata['distance'] = float(dist)
                results.append(metadata)

                if len(results) >= k:
                    break

        return results

    def query_by_image_path(self, image_path: str, k: int = 10,
                           exclude_self: bool = True) -> List[Dict]:
        """Find images similar to an image at given path."""
        image = Image.open(image_path).convert('RGB')
        query_vector = self.extractor.create_feature_vector(image)

        search_k = k + 1 if exclude_self else k
        distances, indices = self.index.search(query_vector, search_k)

        results = []
        for dist, idx in zip(distances, indices):
            if exclude_self and dist < 0.01:
                continue

            metadata = self.index.metadata[idx].copy()
            similarity = self.calculate_similarity_score(dist)

            if similarity >= 0.70:
                metadata['similarity_score'] = similarity
                metadata['distance'] = float(dist)
                results.append(metadata)

            if len(results) >= k:
                break

        return results

    def calculate_similarity_score(self, distance: float) -> float:
        """Convert distance to similarity score (0-1 range)."""
        max_distance = 50000.0
        similarity = max(0.0, 1.0 - (distance / max_distance))
        return float(similarity)

print("✓ ColorQueryEngine loaded!")

✓ ColorQueryEngine loaded!


In [8]:
# ============================================================================
# Main Recommender Class
# ============================================================================

logger = logging.getLogger(__name__)

class ColorRecommender:
    """Main API for color-based image recommendation system."""

    def __init__(self, n_colors: int = 5, quality: int = 10):
        self.n_colors = n_colors
        self.quality = quality

        self.extractor = ColorFeatureExtractor(n_colors=n_colors, quality=quality)
        self.index = ImageIndex(dimension=n_colors * 3)
        self.query_engine = None
        self.dataset = None

    def build_from_directory(self, image_dir: str, recursive: bool = True,
                            max_size: Optional[Tuple[int, int]] = (800, 800)) -> Dict:
        """Build recommendation system from image directory."""
        logger.info("=" * 60)
        logger.info("Building Color-Based Image Recommendation System")
        logger.info("=" * 60)

        start_time = time.time()

        # Load images
        logger.info(f"\n[1/3] Loading images from: {image_dir}")
        self.dataset = load_images_from_directory(image_dir, recursive, max_size)

        if len(self.dataset.images) == 0:
            raise ValueError("No images found in directory")

        logger.info(f"Loaded {len(self.dataset.images)} images")

        # Extract features
        logger.info(f"\n[2/3] Extracting color features...")
        feature_vectors = []
        metadata = []

        for i, img_data in enumerate(self.dataset.images):
            if (i + 1) % 10 == 0:
                logger.info(f"Processing image {i + 1}/{len(self.dataset.images)}")

            try:
                feature_vec = self.extractor.create_feature_vector(img_data['image'])
                feature_vectors.append(feature_vec)

                meta = {
                    'path': img_data['path'],
                    'filename': img_data['filename'],
                    'size': img_data['size']
                }
                metadata.append(meta)
            except Exception as e:
                logger.warning(f"Failed to process {img_data['filename']}: {e}")
                continue

        feature_vectors = np.array(feature_vectors)
        logger.info(f"Extracted features from {len(feature_vectors)} images")

        # Build index
        logger.info(f"\n[3/3] Building FAISS index...")
        self.index.build_index(feature_vectors, metadata)

        # Initialize query engine
        self.query_engine = ColorQueryEngine(self.index, self.extractor)

        elapsed_time = time.time() - start_time

        stats = {
            'total_images': len(self.dataset.images),
            'indexed_images': len(feature_vectors),
            'build_time_seconds': elapsed_time,
            'n_colors': self.n_colors,
            'feature_dimension': self.index.dimension
        }

        logger.info("\n" + "=" * 60)
        logger.info("Build Complete!")
        logger.info(f"Total images: {stats['total_images']}")
        logger.info(f"Indexed images: {stats['indexed_images']}")
        logger.info(f"Build time: {stats['build_time_seconds']:.2f} seconds")
        logger.info("=" * 60 + "\n")

        return stats

    def recommend_by_color(self, color: Union[str, Tuple, List], k: int = 10) -> List[Dict]:
        """Get image recommendations based on color similarity."""
        if self.query_engine is None:
            raise ValueError("Index not built. Call build_from_directory() first.")
        return self.query_engine.query_by_color(color, k)

    def recommend_by_image(self, image_path: str, k: int = 10,
                          exclude_self: bool = True) -> List[Dict]:
        """Get image recommendations based on image similarity."""
        if self.query_engine is None:
            raise ValueError("Index not built. Call build_from_directory() first.")
        return self.query_engine.query_by_image_path(image_path, k, exclude_self)

    def get_statistics(self) -> Dict:
        """Get system statistics."""
        stats = {
            'n_colors': self.n_colors,
            'quality': self.quality
        }

        if self.index:
            stats.update(self.index.get_statistics())

        if self.dataset:
            stats.update(self.dataset.get_statistics())

        return stats

    def get_image_colors(self, image_path: str) -> Dict:
        """Get dominant colors for a specific image."""
        image = Image.open(image_path).convert('RGB')
        return self.extractor.extract_color_palette(image)

print("✓ ColorRecommender loaded!")
print("\n" + "="*60)
print("✅ ALL MODULES LOADED SUCCESSFULLY!")
print("="*60)

✓ ColorRecommender loaded!

✅ ALL MODULES LOADED SUCCESSFULLY!


## 4️⃣ Build Index from Your Images

In [9]:
# Initialize and build the recommendation system
print("🔨 Building index from your images...")
print("This will take ~1-2 seconds per image.\n")

recommender = ColorRecommender(n_colors=5, quality=10)

stats = recommender.build_from_directory(
    image_dir=IMAGE_DIR,
    recursive=True,
    max_size=(800, 800)
)

print("\n✅ INDEX READY!")
print(f"   Total images indexed: {stats['indexed_images']}")
print(f"   Build time: {stats['build_time_seconds']:.2f} seconds")
print(f"   Feature dimension: {stats['feature_dimension']}")

🔨 Building index from your images...
This will take ~1-2 seconds per image.


✅ INDEX READY!
   Total images indexed: 59
   Build time: 15.78 seconds
   Feature dimension: 15


## 5️⃣ Interactive Color Search! 🎨

In [10]:
# Results display function
def display_results(results, query_color, query_time_ms):
    """Display search results in matplotlib grid."""
    n_results = len(results)
    cols = 4
    rows = (n_results + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(16, 4*rows))

    if rows == 1 and cols == 1:
        axes = np.array([[axes]])
    elif rows == 1:
        axes = axes.reshape(1, -1)
    elif cols == 1:
        axes = axes.reshape(-1, 1)

    axes = axes.flatten()

    for idx, result in enumerate(results):
        try:
            img = Image.open(result['path'])
            axes[idx].imshow(img)
            axes[idx].axis('off')

            similarity = result['similarity_score'] * 100
            filename = result['filename'][:30] + '...' if len(result['filename']) > 30 else result['filename']

            axes[idx].set_title(
                f"{filename}\n✓ {similarity:.1f}% match",
                fontsize=9,
                pad=5,
                fontweight='bold' if similarity > 80 else 'normal'
            )
        except Exception as e:
            axes[idx].text(0.5, 0.5, f'Error loading\n{result["filename"]}',
                          ha='center', va='center', fontsize=8)
            axes[idx].axis('off')

    for idx in range(n_results, len(axes)):
        axes[idx].axis('off')

    fig.suptitle(
        f"🎨 Query Color: {query_color}  |  📊 {n_results} Results  |  ⚡ {query_time_ms:.2f}ms",
        fontsize=14,
        fontweight='bold',
        y=0.98
    )

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

print("✓ Display function ready!")

✓ Display function ready!


In [11]:
# Interactive color search widget
color_picker = widgets.ColorPicker(
    concise=False,
    description='🎨 Pick Color:',
    value='#FF5733',
    disabled=False,
    style={'description_width': 'initial'}
)

k_slider = widgets.IntSlider(
    value=12,
    min=1,
    max=50,
    step=1,
    description='🔢 Results:',
    style={'description_width': 'initial'}
)

search_btn = widgets.Button(
    description='🔍 Search Images',
    button_style='success',
    tooltip='Click to search for similar images',
    icon='search',
    layout=widgets.Layout(width='200px', height='40px')
)

output_area = widgets.Output()

def on_search_clicked(btn):
    with output_area:
        clear_output(wait=True)

        color = color_picker.value
        k = k_slider.value

        print(f"🔍 Searching for {k} images with color: {color}")

        start_time = time.time()
        results = recommender.recommend_by_color(color, k=k)
        query_time = (time.time() - start_time) * 1000

        if not results:
            print("❌ No results found!")
            return

        print(f"✓ Found {len(results)} results in {query_time:.2f}ms\n")
        display_results(results, color, query_time)

search_btn.on_click(on_search_clicked)

display(HTML("<h2>🎨 Color-Based Image Search</h2>"))
display(widgets.VBox([
    widgets.HBox([color_picker, k_slider]),
    search_btn,
    output_area
], layout=widgets.Layout(padding='10px')))

print("\n👆 Pick a color and click 'Search Images'!")


👆 Pick a color and click 'Search Images'!


## 6️⃣ Advanced Features

In [ ]:
# Direct color search (without widget)
# Uncomment and run to search directly:

# color = '#FF5733'  # Change this!
# results = recommender.recommend_by_color(color, k=12)
# display_results(results, color, 0)

In [ ]:
# Analyze colors in a specific image
# Uncomment and modify path:

# image_path = 'images/your_image.jpg'  # Change this!
# colors = recommender.get_image_colors(image_path)
#
# print("Dominant Colors:")
# for i, (rgb, hex_code) in enumerate(zip(colors['rgb_colors'], colors['hex_colors'])):
#     print(f"  Color {i+1}: {hex_code} - RGB{rgb}")

In [ ]:
# Reverse image search (find similar images)
# Uncomment and modify path:

# query_image = 'images/your_image.jpg'  # Change this!
# similar = recommender.recommend_by_image(query_image, k=10, exclude_self=True)
#
# print(f"Images similar to {query_image}:")
# for img in similar:
#     print(f"  {img['filename']}: {img['similarity_score']*100:.1f}% match")

In [ ]:
# View system statistics
stats = recommender.get_statistics()

print("="*60)
print("📊 SYSTEM STATISTICS")
print("="*60)
for key, value in stats.items():
    print(f"  {key}: {value}")
print("="*60)

## 💡 Tips & Usage

### Color Search:
```python
# By HEX code
results = recommender.recommend_by_color('#FF5733', k=10)

# By RGB tuple
results = recommender.recommend_by_color((255, 87, 51), k=10)
```

### Popular Colors to Try:
- **Vibrant Red**: `#FF5733`
- **Ocean Blue**: `#3498DB`
- **Forest Green**: `#2ECC71`
- **Sunset Orange**: `#FF8C42`
- **Royal Purple**: `#9B59B6`
- **Hot Pink**: `#E91E63`

### Notes:
- Results are filtered at 70% similarity threshold
- FAISS uses exact L2 distance (no approximation)
- Query time typically 5-50ms depending on index size
- All code is embedded in this notebook - no external files needed!

---

**Enjoy your color-based image search! 🎨**
